In [1]:
import pandas as pd
import numpy  as np
###
# Null date conversion fun
def conv(val):
    if not val:
        return 0    
    try:
        return np.int(val)
    except:        
        return np.int(0)

In [2]:
import pandas as pd
import numpy  as np

#################################################
# Principals
#################################################

P = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                )
P["cat_cln"] = np.where (P['category'].str.contains('actor') | P['category'].str.contains('actress')
                         ,'actor'
                         ,P['category'])
P["cat_cln"] = np.where (P['category'].str.contains('self') | P['category'].str.contains('archive_footage')
                         ,'self',P['cat_cln'])


P.head(55)

,tconst,ordering,nconst,category,job,characters,cat_cln
0,tt0000001,1,nm1588970,self,\N,"[""Self""]",self
1,tt0000001,2,nm0005690,director,\N,\N,director
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N,cinematographer
3,tt0000002,1,nm0721526,director,\N,\N,director
4,tt0000002,2,nm1335271,composer,\N,\N,composer
5,tt0000003,1,nm0721526,director,\N,\N,director
6,tt0000003,2,nm1770680,producer,producer,\N,producer
7,tt0000003,3,nm1335271,composer,\N,\N,composer
8,tt0000003,4,nm5442200,editor,\N,\N,editor
9,tt0000004,1,nm0721526,director,\N,\N,director


In [3]:
import pandas as pd
import numpy  as np
#################################################
# Titles
#################################################

def conv(val):
    if not val:
        return 0    
    try:
        return np.int(val)
    except:        
        return np.int(0)

T = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t'
                 , encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                #, error_bad_lines=False
                #, dtype={'startYear': int }
                 , converters={'startYear':conv,'endYear':conv}                
                ) 
T["flg_doc"]           = np.where (T['genres'].str.contains('Documentary'),1,0)
T["primaryTitle_year"] = T['primaryTitle']  + ' ('+  T['startYear'].map(str)  + ')'
## filter movie 
T = T.loc[T.titleType == 'movie', :]
T.head(10)

C:\Users\cw13\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year
498,tt0000502,movie,Bohemios,Bohemios,0,1905,0,100,\N,0,Bohemios (1905)
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,0,70,"Action,Adventure,Biography",0,The Story of the Kelly Gang (1906)
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,0,90,Drama,0,The Prodigal Son (1907)
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,0,\N,Drama,0,Robbery Under Arms (1907)
625,tt0000630,movie,Hamlet,Amleto,0,1908,0,\N,Drama,0,Hamlet (1908)
668,tt0000675,movie,Don Quijote,Don Quijote,0,1908,0,\N,Drama,0,Don Quijote (1908)
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,0,120,"Adventure,Fantasy",0,The Fairylogue and Radio-Plays (1908)
783,tt0000793,movie,Andreas Hofer,Andreas Hofer,0,1909,0,\N,Drama,0,Andreas Hofer (1909)
804,tt0000814,movie,La bocana de Mar Chica,La bocana de Mar Chica,0,1909,0,\N,\N,0,La bocana de Mar Chica (1909)
828,tt0000838,movie,A Cultura do Cacau,A Cultura do Cacau,0,1909,0,\N,\N,0,A Cultura do Cacau (1909)


In [4]:
var_list = ('tconst','primaryTitle' , 'titleType')
#D = newdf.loc[:,('const','title', 'dat_per', 'title_type')]
D = T.loc[:, var_list]
D.head()
pd.get_dummies(D, columns=['titleType']).head(3)


,tconst,primaryTitle,titleType_movie
498,tt0000502,Bohemios,1
570,tt0000574,The Story of the Kelly Gang,1
587,tt0000591,The Prodigal Son,1


In [5]:
#################################################
# Names
#################################################
    
N = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                 , converters={'birthYear':conv,'deathYear':conv}         
                )


#N["primaryName_year"] = N['primaryName']  + ' ('+  N['birthYear'].map(str) + ' - ' +  N['deathYear'].map(str)  + ')'

N["primaryName_year"] = N['primaryName']  + np.where(N['birthYear']==0, ' ',  ' ('+  N['birthYear'].map(str) + ' - '  +  np.where (N['deathYear'] == 0, ' ',  N['deathYear'].map(str)) + ')')
N.head(3)


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,primaryName_year
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0031983,tt0050419",Fred Astaire (1899 - 1987)
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0071877",Lauren Bacall (1924 - 2014)
2,nm0000003,Brigitte Bardot,1934,0,"actress,soundtrack,music_department","tt0054452,tt0049189,tt0056404,tt0057345",Brigitte Bardot (1934 - )


In [6]:
#################################################
# Ratings
#################################################
R = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                )
R.tail(3)

,tconst,averageRating,numVotes
1244076,tt9916730,8.7,6
1244077,tt9916766,6.7,19
1244078,tt9916778,7.2,34


In [7]:
#################################################
# Join
#################################################
from datetime import datetime
             
imdb = pd.merge(    P
                  , T 
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )
imdb = pd.merge(     N.loc[:, ('nconst','primaryProfession', 'primaryName', 'birthYear', 'deathYear', 'primaryName_year')]
                  , imdb
                  , how = 'inner', left_on="nconst", right_on="nconst"
                 )
imdb = pd.merge(     R.loc[:, ('tconst','averageRating', 'numVotes')]
                  , imdb
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )

## imdb["birthYear_cln"] = np.where (imdb["birthYear"].str.contains('N')   
##                          , "3999"
##                          ,imdb["birthYear"])
##imdb["deathYear_cln"] = np.where (imdb["deathYear"].str.contains('N')   
##                         , "3999"
##                          ,imdb["deathYear"]) 
##imdb["startYear_cln"] = np.where (imdb["startYear"].str.contains('N')   
##                       , "1888"
##                          ,imdb["birthYear_cln"]) 
##imdb["age_crew"]            = int(imdb["birthYear_cln"]) -  int(imdb["startYear_cln"])
imdb["last_refresh"]        = datetime.now().strftime('%Y%m%d%H%M%S')
print(P.shape, imdb.shape)


(50302221, 7) (2480553, 25)


In [8]:
imdb.head(100)

,tconst,averageRating,numVotes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,ordering,...,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year,last_refresh
0,tt0000502,4.5,14,nm0063413,"director,cinematographer,writer",Ricardo de Baños,1884,1939,Ricardo de Baños (1884 - 1939),3,...,Bohemios,Bohemios,0,1905,0,100,\N,0,Bohemios (1905),20220515121715
1,tt0000502,4.5,14,nm0215752,actor,Antonio del Pozo,0,0,Antonio del Pozo,1,...,Bohemios,Bohemios,0,1905,0,100,\N,0,Bohemios (1905),20220515121715
2,tt0000502,4.5,14,nm0252720,actor,El Mochuelo,0,0,El Mochuelo,2,...,Bohemios,Bohemios,0,1905,0,100,\N,0,Bohemios (1905),20220515121715
3,tt0000502,4.5,14,nm0657268,"soundtrack,writer",Miguel de Palacios,1863,1920,Miguel de Palacios (1863 - 1920),4,...,Bohemios,Bohemios,0,1905,0,100,\N,0,Bohemios (1905),20220515121715
4,tt0000502,4.5,14,nm0675388,"soundtrack,writer",Guillermo Perrín,1875,1923,Guillermo Perrín (1875 - 1923),5,...,Bohemios,Bohemios,0,1905,0,100,\N,0,Bohemios (1905),20220515121715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,tt0001115,4.6,19,nm0630641,actress,Marie Niedermann,1880,1967,Marie Niedermann (1880 - 1967),10,...,Ansigttyven I,Ansigttyven I,0,1910,0,\N,Crime,0,Ansigttyven I (1910),20220515121715
96,tt0001115,4.6,19,nm1834296,"writer,actor",Axel Garde,1876,1958,Axel Garde (1876 - 1958),7,...,Ansigttyven I,Ansigttyven I,0,1910,0,\N,Crime,0,Ansigttyven I (1910),20220515121715
97,tt0001116,4.7,18,nm0064953,actor,Philip Bech,1869,1928,Philip Bech (1869 - 1928),1,...,Ansigttyven II,Ansigttyven II,0,1910,0,\N,\N,0,Ansigttyven II (1910),20220515121715
98,tt0001116,4.7,18,nm0085066,actor,Aage Bjørnbak,1875,1913,Aage Bjørnbak (1875 - 1913),2,...,Ansigttyven II,Ansigttyven II,0,1910,0,\N,\N,0,Ansigttyven II (1910),20220515121715


In [9]:
#################################################
# add row number forfilmmakers with more than one release per year
#################################################
imdb['RN'] = imdb.sort_values(['nconst', 'tconst'], ascending=[True, False]) \
             .groupby(['nconst', 'startYear']) \
             .cumcount() + 1
imdb.tail(100)

,tconst,averageRating,numVotes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,ordering,...,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year,last_refresh,RN
2480453,tt9915436,7.4,5,nm10537887,writer,Joana Kfuri,0,0,Joana Kfuri,3,...,Vida em Movimento,0,2019,0,70,Documentary,1,Vida em Movimento (2019),20220515121715,1
2480454,tt9915436,7.4,5,nm10537888,producer,Marina Costa Manso,0,0,Marina Costa Manso,5,...,Vida em Movimento,0,2019,0,70,Documentary,1,Vida em Movimento (2019),20220515121715,1
2480455,tt9915436,7.4,5,nm3644375,"editor,writer,script_department",Selma Perez,0,0,Selma Perez,4,...,Vida em Movimento,0,2019,0,70,Documentary,1,Vida em Movimento (2019),20220515121715,1
2480456,tt9915436,7.4,5,nm4704814,"cinematographer,director,producer",Humberto Bassanelli,0,0,Humberto Bassanelli,6,...,Vida em Movimento,0,2019,0,70,Documentary,1,Vida em Movimento (2019),20220515121715,1
2480457,tt9915436,7.4,5,nm6119329,"director,writer,miscellaneous",Eduardo Rajabally,0,0,Eduardo Rajabally,2,...,Vida em Movimento,0,2019,0,70,Documentary,1,Vida em Movimento (2019),20220515121715,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480548,tt9916730,8.7,6,nm13233318,"actor,production_manager",Ganesh Vasant Patil,0,0,Ganesh Vasant Patil,3,...,6 Gunn,0,2017,0,116,\N,0,6 Gunn (2017),20220515121715,1
2480549,tt9916730,8.7,6,nm1957275,"cinematographer,camera_department,producer",Suresh Deshmane,0,0,Suresh Deshmane,8,...,6 Gunn,0,2017,0,116,\N,0,6 Gunn (2017),20220515121715,1
2480550,tt9916730,8.7,6,nm4852679,actor,Bhushan Pradhan,0,0,Bhushan Pradhan,4,...,6 Gunn,0,2017,0,116,\N,0,6 Gunn (2017),20220515121715,1
2480551,tt9916730,8.7,6,nm6096005,actor,Devadhar Archit,0,0,Devadhar Archit,1,...,6 Gunn,0,2017,0,116,\N,0,6 Gunn (2017),20220515121715,1


In [10]:
#################################################
#remove rows not needed 
#################################################

X = imdb[imdb['category']!= "production_designer"] 
X = imdb[imdb['category']!= "archive_sound"] 
#X = X[X['category']!= "archive_footage"]
print(X.shape, imdb.shape)

#X.sort_values(by=['tconst', 'averageRating'], ascending=False)
#################################################
# export to file
#################################################
X.to_csv('data/imdb.csv'  ,encoding='utf-8')

(2480508, 26) (2480553, 26)


In [67]:
#################################################
# Distinct NCONST aggregation
#################################################
import numpy as np
N_avg = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.mean]                   
                   , values = ["averageRating"]
                   , fill_value = 0
             )

N_sum = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.sum]                   
                   , values = [ "numVotes"]
                   , fill_value = 0
             )
N_cnt = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ 'count']  
                   , values = ["tconst"]
                   , fill_value = 0
             )

N_unique = pd.merge(   N_sum
                     , N_avg 
                     , how = 'inner', left_on=  [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )
N_unique = pd.merge(   N_unique
                     , N_cnt
                     , how = 'inner', left_on= [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )

In [74]:
#N_unique.head(66)
N_cnt.head()
N_unique.to_csv('data/nconst.csv' , header=None , encoding='utf-8')